<a href="https://colab.research.google.com/github/hrsh668/ml/blob/main/cnn_and_dl_for_image_classification_of_cifar10_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers
import sklearn.model_selection
from tensorflow.keras.models import Sequential
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D
import random
import math
# importing libraries


import tensorflow as tf
device_name = tf.test.gpu_device_name()
if len(device_name) > 0:
    print("Found GPU at: {}".format(device_name))
else:
    device_name = "/device:CPU:0"
    print("No GPU, using {}.".format(device_name))
# setting up the GPU feature of colab

No GPU, using /device:CPU:0.


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Load CIFAR-10 dataset
cifar10_dataset, cifar10_info = tfds.load(name='cifar10', with_info=True, as_supervised=True)

cifar10_train, cifar10_test = cifar10_dataset['train'], cifar10_dataset['test']

# Assume you want to use 10% of the training data for validation
num_validation_samples = 0.1 * cifar10_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = cifar10_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)


def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0  # Normalize pixel values to the range [0, 1]
    return image, label

scaled_train_and_validation_data = cifar10_train.map(scale)

test_data = cifar10_test.map(scale)
# SCALING IMAGES

BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)
# SHUFFLING IMAGES TO REDUCE OVERFITTING

validation_data = scaled_train_and_validation_data.take(num_validation_samples)
train_data = scaled_train_and_validation_data.skip(num_validation_samples)
# COMPLETING THE SPLIT OF DATASET


BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)

test_data = test_data.batch(num_test_samples)
validation_inputs, validation_targets = next(iter(validation_data))
# BATCHING DATA TO ENSURE DATASETS(validation/training/test are iterable)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incompleteH13SD6/cifar10-train.tfrecord*...:   0%|          …

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incompleteH13SD6/cifar10-test.tfrecord*...:   0%|          |…

Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


In [ ]:
#Convolutional Neural Network Layer
def build_cnn(n_cnn_layers = 1, n_hidden_layers=1, n_neurons=10, drop_out_pctg = .2, hidden_layer_function='relu', initial_filter_size=7, n_filters_initial=16, batch_size=1, pool_size=2):
      num_classes = 10
      NUM_EPOCHS = 100
      early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)
      model = tf.keras.models.Sequential()
      model.add(tf.keras.layers.Conv2D(n_filters_initial, 7, padding='same', activation='relu', input_shape=(32,32,3)))
      model.add(tf.keras.layers.MaxPooling2D(pool_size=pool_size))
      # initial convolutional/pooling layers added
      z = 2
      for i in range(n_cnn_layers):
          model.add(tf.keras.layers.Conv2D(z*n_filters_initial, initial_filter_size, padding='same', activation='relu'))
          model.add(tf.keras.layers.Conv2D(z*n_filters_initial, initial_filter_size, padding='same', activation='relu'))
          z*=2
          model.add(tf.keras.layers.MaxPooling2D(pool_size=pool_size))
          model.add(tf.keras.layers.Dropout(drop_out_pctg))
          # additional 2 convolutional+pooling layers added
          # dropout layer added
          # convolutional layers have double the previous layers
      model.add(tf.keras.layers.Flatten())
      for j in range(n_hidden_layers):
        model.add(tf.keras.layers.Dense(n_neurons, activation=hidden_layer_function))
      model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))
      # addditional x # of hidden layers added
      # final classification softmax layer added
      model.compile(optimizer='adam',
                    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                    metrics=['accuracy'])
      history  = model.fit(train_data, epochs = NUM_EPOCHS, callbacks=[early_stopping], validation_data=(validation_inputs, validation_targets),
            validation_steps=10, batch_size=batch_size, verbose=2)
      final_accuracy = history.history['accuracy'][-1]  # Extracting the final accuracy

      return final_accuracy
# the function builds a cnn model, compiles it, trains it, and returns the final accuracy

In [ ]:
hidden_layer_functions = ['relu', 'softmax']
initial_filter_number = [16, 32, 64]
batch_sizes = [2,4,8,16,32]
best_acc = 0
best_param = 0
all_result = {}
# storage variables and possible parameter values declared

for i in range(1000):
    n_cnn_layers = random.randint(0, 3)
    n_hidden_layers = random.randint(1, 3)
    n_neurons = random.randint(10, 100)
    stride = random.randint(1, 3)
    pool_size = random.randint(2, 3)
    initial_filter_size = random.randint(1, 3)
    # some integer range inputs to build cnn are randomly selected

    drop_out_pctg = random.uniform(0.5, 0.8)
    # floating point range input randomly selected

    n_filters_initial = random.choice(initial_filter_number)
    batch_size = random.choice(batch_sizes)
    hidden_layer_function = random.choice(hidden_layer_functions)
    # input values randomly chosen from lists

    final_acc = build_cnn(n_cnn_layers, n_hidden_layers, n_neurons, drop_out_pctg, hidden_layer_function, initial_filter_size, n_filters_initial, batch_size, pool_size)
    # final_acc seto to equal build_cnn with the randomly selected parameter inputs

    if final_acc>best_acc:
      best_acc = final_acc
      best_param = [n_cnn_layers, n_hidden_layers, n_neurons, drop_out_pctg, n_filters_initial,initial_filter_size, batch_size, hidden_layer_function, pool_size]
      # if the curren acc is better than best_acc, best_acc is replaced

    all_result[i] = [final_acc, [n_cnn_layers, n_hidden_layers, n_neurons, drop_out_pctg, n_filters_initial,initial_filter_size, batch_size, hidden_layer_function, pool_size]]

print(best_acc)
print(best_param)
# best acc and parameter combination printed


# in each of the 1000 iteration, random values are selected for the build_cnn value
# the best value and combination is returned

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5729: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


450/450 - 27s - loss: 2.2114 - accuracy: 0.1509 - val_loss: 2.0722 - val_accuracy: 0.2094 - 27s/epoch - 61ms/step
Epoch 2/100
450/450 - 22s - loss: 2.1596 - accuracy: 0.1681 - val_loss: 2.0495 - val_accuracy: 0.2094 - 22s/epoch - 48ms/step
Epoch 3/100
450/450 - 20s - loss: 2.1484 - accuracy: 0.1818 - val_loss: 2.0430 - val_accuracy: 0.2500 - 20s/epoch - 43ms/step
Epoch 4/100
450/450 - 20s - loss: 2.1351 - accuracy: 0.1894 - val_loss: 2.0286 - val_accuracy: 0.2281 - 20s/epoch - 45ms/step
Epoch 5/100
450/450 - 21s - loss: 2.1245 - accuracy: 0.1959 - val_loss: 2.0305 - val_accuracy: 0.2250 - 21s/epoch - 46ms/step
Epoch 6/100
450/450 - 20s - loss: 2.1190 - accuracy: 0.1996 - val_loss: 2.0221 - val_accuracy: 0.2469 - 20s/epoch - 44ms/step
Epoch 7/100
450/450 - 22s - loss: 2.1148 - accuracy: 0.2012 - val_loss: 2.0202 - val_accuracy: 0.2438 - 22s/epoch - 48ms/step
Epoch 8/100
450/450 - 21s - loss: 2.1132 - accuracy: 0.2045 - val_loss: 2.0170 - val_accuracy: 0.2500 - 21s/epoch - 47ms/step
Epoc

In [ ]:
def build_dl_model(n_hidden=1, n_neurons=2, batch_size=2, hidden_layer_function='relu'):
  NUM_EPOCHS = 100
  early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

  with tf.device(device_name):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape = (32,32,3)))
    model.add(tf.keras.layers.Flatten())
    # images flattened into arrays

    for layer in range(n_hidden):
      model.add(tf.keras.layers.Dense(n_neurons, activation=hidden_layer_function))
      # variable number of hidden layers of same size and function added to model

    model.add(tf.keras.layers.Dense(output_size, activation='softmax'))
    # softmax classification layer added to function

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    history  = model.fit(train_data, epochs = NUM_EPOCHS, callbacks=[early_stopping], validation_data=(validation_inputs, validation_targets),
            validation_steps=10, batch_size=batch_size, verbose=0)
    final_accuracy = history.history['accuracy'][-1]  # Compiling and then extracting the final accuracy

    return final_accuracy


In [ ]:
hidden_layer_functions = ['relu', 'softmax', 'sigmoid']
batch_sizes = [2,4,8,16,32]
best_acc = 0
best_hyp = {'n_hidden':0, 'n_neurons':0, 'batch_size':2, 'hidden_layer_function':'h'}
all_acc_and_hyp = {}
# storage variables and input ranges declared

for i in range(1000):
  n_neurons = random.randint(10, 784)
  n_hidden = random.randint(2, 8)
  batch_size = random.choice(batch_sizes)
  hidden_layer_function = random.choice(hidden_layer_functions)
  # input values randomly selected

  final_training_accuracy = build_dl_model(n_neurons, n_hidden, batch_size, hidden_layer_function)
  # build dl model called

  if final_training_accuracy > best_acc:
    best_acc = final_training_accuracy
    best_hyp = [n_neurons, n_hidden, batch_size, hidden_layer_function]
  all_acc_and_hyp[i] = [final_training_accuracy,[n_neurons, n_hidden, batch_size, hidden_layer_function]]
  print(best_acc)
  print(best_hyp)
  # best accuracy calculated and return

  # this random search runs 1000 times to finetune to get the best accuracy and hyperparameters